In [50]:
%run data.ipynb

In [51]:
import pandas as pd

df_paypal_reconciliation = df_paypal.copy()
df_paypal_reconciliation.head()

,Date,Time,TimeZone,Name,Type,Status,Currency,Gross,Fee,Net,From Email Address,To Email Address,Transaction ID,Address Status,Sales Tax,Reference Txn ID,Invoice Number,Receipt ID,Balance,Contact Phone Number,Subject,Note,Balance Impact,Datetime
0,08/28/2023,03:14:11,PDT,Sven Amiet,Subscription Payment,Completed,USD,10.0,-0.99,9.01,sven.amiet@gmail.com,host+c6@opencollective.org,6M040519WW138784Y,Non-Confirmed,0.0,I-W0G0VXY7746L,NaN,NaN,35984.92,0795475664,Monthly financial contribution to OSArch - Ope...,NaN,Credit,2023-08-28 03:14:11-07:00
1,08/28/2023,03:14:46,PDT,Kaleb Silva,Subscription Payment,Completed,USD,4.0,-0.63,3.37,zorchedscum@gmail.com,host+c6@opencollective.org,9UN606960R585570S,Non-Confirmed,0.0,I-2C2YMFYNLHUD,NaN,NaN,35988.29,6052091606,Monthly financial contribution to .fmbot (.fmb...,NaN,Credit,2023-08-28 03:14:46-07:00
2,08/28/2023,03:15:23,PDT,Realitycraft,Subscription Payment,Completed,USD,5.0,-0.66,4.34,lightlodges@gmail.com,host+c6@opencollective.org,7PP64013M1026212Y,Non-Confirmed,0.0,I-BWWB3R6V49EA,NaN,NaN,35992.63,3109531087,Monthly financial contribution to Open Metaver...,NaN,Credit,2023-08-28 03:15:23-07:00
3,08/28/2023,03:15:31,PDT,ABDUL SOOMRO,Subscription Payment,Completed,USD,5.0,-0.66,4.34,info@programmatic.llc,host+c6@opencollective.org,4JV60907AA934033X,Non-Confirmed,0.0,I-6FX1VNTVLCYS,NaN,NaN,35996.97,6142865339,Monthly financial contribution to Mocha (Backers),NaN,Credit,2023-08-28 03:15:31-07:00
4,08/28/2023,03:18:23,PDT,Sven Amiet,Subscription Payment,Completed,USD,10.0,-0.99,9.01,sven.amiet@gmail.com,host+c6@opencollective.org,0BM73433YM818282N,Non-Confirmed,0.0,I-P7VSXAH0HV8C,NaN,NaN,36005.98,0795475664,Monthly financial contribution to OpenSourceBI...,NaN,Credit,2023-08-28 03:18:23-07:00


In [52]:
df_platform_paypal_related = df_platform[(df_platform['paymentMethod.service'] == 'PAYPAL') | (df_platform['payoutMethod.type'] == 'PAYPAL')]

In [64]:
df_platform_paypal = pd.merge(df_platform_paypal_related.dropna(subset=['merchantId']), df_paypal_reconciliation, left_on='merchantId', right_on='Transaction ID', how='left').dropna(subset=['Transaction ID'])
print('Successfully merged {} rows out of {}'.format(len(df_platform_paypal), len(df_platform_paypal_related)))

# First Pass, joining with IDs
df_left_platform = df_platform_paypal_related[~df_platform_paypal_related['id'].isin(df_platform_paypal['id'])]
df_left_paypal = df_paypal_reconciliation[~df_paypal_reconciliation['Transaction ID'].isin(df_platform_paypal['Transaction ID'])]
df_left_paypal = df_left_paypal[~(df_left_paypal['Transaction ID'].isin(df_platform_paypal['Reference Txn ID']))]

# Second Pass, joining with dates and amounts
from utils import matchByDateAndValue
merged_df2 = matchByDateAndValue(df_left_paypal, ['Datetime', 'Gross'], df_left_platform, ['createdAt', 'amountInHostCurrency.value'] , '2 hours', 0.05).dropna(subset=['df2key'])
df_platform_paypal = pd.concat([df_platform_paypal, merged_df2.loc[:, merged_df2.columns!='df2key']], ignore_index=True)
print('Successfully merged {} rows out of {} from Value and Date'.format(len(merged_df2), len(df_platform_paypal_related)))


print(f"Platform transactions that have a corresponding PayPal transaction: {len(df_platform_paypal)}")
print(f"Platform transactions without a corresponding PayPal transaction: {len(df_left_platform)}")
print(f"Percentage of Platform transactions that have a corresponding PayPal transaction: {round(len(df_platform_paypal)/len(df_platform_paypal_related)*100, 2)}%")
print(f"PayPal transactions without a corresponding platform transaction: {len(df_left_paypal)}")
print(f"Percentage of PayPal transactions that have a corresponding Platform transaction: {round(len(df_platform_paypal)/len(df_paypal_reconciliation)*100, 2)}%")


Successfully merged 59986 rows out of 60016
Successfully merged 1 rows out of 60016 from Value and Date
Platform transactions that have a corresponding PayPal transaction: 59987
Platform transactions without a corresponding PayPal transaction: 30
Percentage of Platform transactions that have a corresponding PayPal transaction: 99.95%
PayPal transactions without a corresponding platform transaction: 2265
Percentage of PayPal transactions that have a corresponding Platform transaction: 94.57%


In [67]:
df_left_paypal['Datetime'].value_counts()

Datetime
2023-12-26 11:02:03-08:00    13
2023-11-14 10:02:19-08:00    12
2023-08-03 23:01:50-07:00    12
2023-06-23 10:01:55-07:00    12
2023-03-28 10:03:44-07:00    10
2023-11-14 10:02:18-08:00     9
2023-02-16 09:01:52-08:00     9
2023-10-31 10:01:45-07:00     9
2023-10-17 09:01:52-07:00     8
2023-02-09 05:01:12-08:00     8
2023-11-02 10:02:04-07:00     8
2023-08-31 09:01:15-07:00     8
2023-04-25 09:02:08-07:00     8
2023-09-12 09:01:18-07:00     7
2023-05-02 23:00:35-07:00     7
2023-10-24 09:01:49-07:00     7
2023-10-12 09:01:39-07:00     7
2023-01-09 15:01:01-08:00     7
2023-06-08 09:00:59-07:00     7
2023-08-02 09:01:13-07:00     7
2023-09-12 09:01:23-07:00     6
2023-10-12 09:01:40-07:00     6
2023-11-14 10:02:14-08:00     6
2023-02-02 11:00:50-08:00     6
2023-01-05 12:00:56-08:00     6
2023-06-02 09:00:55-07:00     6
2023-12-26 11:02:02-08:00     6
2023-07-18 09:01:07-07:00     6
2023-06-16 10:01:08-07:00     6
2023-03-14 10:04:47-07:00     6
2023-05-23 09:01:15-07:00     6

In [54]:
# merged_df[['id', 'Transaction ID']].to_csv('platform_paypal_reconciliation.csv', index=False)

In [55]:
pd.set_option('display.max_rows', None)
print(df_left_paypal.value_counts('Balance Impact'))
print(df_left_paypal.value_counts('Status'))
df_left_paypal.value_counts('Type')

Balance Impact
Credit    1922
Debit      343
Name: count, dtype: int64
Status
Completed    2148
Denied         55
Pending        54
Unclaimed       8
Name: count, dtype: int64


Type
Reversal of General Account Hold               1655
General Currency Conversion                     292
Cancellation of Hold for Dispute Resolution      55
Hold on Balance for Dispute Investigation        53
Payment Refund                                   46
Hold on Available Balance                        34
Chargeback                                       29
Mass Pay Payment                                 20
Subscription Payment                             16
User Initiated Currency Conversion               16
General Payment                                  14
Express Checkout Payment                         11
Mass Pay Reversal                                 9
General Withdrawal                                5
Chargeback Reversal                               5
Mobile Payment                                    3
Payment Reversal                                  1
Bank Deposit to PP Account                        1
Name: count, dtype: int64

In [56]:
df_left_paypal[df_left_paypal['Type'] == 'Express Checkout Payment']

,Date,Time,TimeZone,Name,Type,Status,Currency,Gross,Fee,Net,From Email Address,To Email Address,Transaction ID,Address Status,Sales Tax,Reference Txn ID,Invoice Number,Receipt ID,Balance,Contact Phone Number,Subject,Note,Balance Impact,Datetime
30352,12/31/2023,15:02:30,PST,Soham Koradia,Express Checkout Payment,Completed,USD,20.0,-1.19,18.81,sohamkoradia@gmail.com,host+c6@opencollective.com,653155676A0995112,Non-Confirmed,0.0,13P94981YN705694L,Contribution #724348,NaN,160939.18,4083292724,NaN,NaN,Credit,2023-12-31 15:02:30-08:00
30355,12/31/2023,15:55:56,PST,Andreas Boehlk Computer-Service,Express Checkout Payment,Completed,USD,50.0,-2.99,47.01,computer@boehlk.com,host+c6@opencollective.com,29E56415EE019194K,Non-Confirmed,0.0,9XV91712XB563113L,Contribution #724369,NaN,160986.19,51412999511,NaN,NaN,Credit,2023-12-31 15:55:56-08:00
30358,12/31/2023,15:57:27,PST,Tomo Makabe,Express Checkout Payment,Completed,USD,5.0,-0.74,4.26,mkbtm1968@gmail.com,host+c6@opencollective.com,68C0530318387690X,Non-Confirmed,0.0,6J597435RX042300U,Contribution #724370,NaN,160990.45,9024780721,NaN,NaN,Credit,2023-12-31 15:57:27-08:00
30361,12/31/2023,17:21:16,PST,Timothy Schwanitz,Express Checkout Payment,Completed,USD,50.0,-2.24,47.76,tschwanitz2017@gmail.com,host+c6@opencollective.com,9KX42855RC526414N,Non-Confirmed,0.0,1J086036TA854221S,Contribution #724391,NaN,161038.21,6099331552,NaN,NaN,Credit,2023-12-31 17:21:16-08:00
30366,12/31/2023,17:49:09,PST,marcin adamczyk,Express Checkout Payment,Completed,USD,10.0,-0.84,9.16,marcin_a@icloud.com,host+c6@opencollective.com,1YG60006GK973541L,Non-Confirmed,0.0,0W520166SY387120R,Contribution #724398,NaN,161055.89,8472241654,NaN,NaN,Credit,2023-12-31 17:49:09-08:00
30370,12/31/2023,18:23:03,PST,Joseph Marks,Express Checkout Payment,Completed,USD,5.0,-0.66,4.34,joseph.b.marks@pm.me,host+c6@opencollective.com,7J396175TV2976503,Non-Confirmed,0.0,3435285313790542M,Contribution #724405,NaN,161066.91,3373443443,NaN,NaN,Credit,2023-12-31 18:23:03-08:00
30373,12/31/2023,19:18:24,PST,Jeff Sikes,Express Checkout Payment,Completed,USD,20.0,-1.19,18.81,box464@gmail.com,host+c6@opencollective.com,9G469989PG116635V,Non-Confirmed,0.0,1MX524721S9472436,Contribution #724419,NaN,161085.72,2149143228,NaN,NaN,Credit,2023-12-31 19:18:24-08:00
30377,12/31/2023,20:32:09,PST,Brian Osorio,Express Checkout Payment,Completed,USD,20.0,-1.19,18.81,spiked.arcadia@gmail.com,host+c6@opencollective.com,23N73181WV3394513,Non-Confirmed,0.0,47357670LR870624K,Contribution #724433,NaN,161219.85,5735288468,NaN,NaN,Credit,2023-12-31 20:32:09-08:00
30380,12/31/2023,20:35:07,PST,Brian Osorio,Express Checkout Payment,Completed,USD,20.0,-1.19,18.81,spiked.arcadia@gmail.com,host+c6@opencollective.com,8VM670366Y291884Y,Non-Confirmed,0.0,1NY3870732944841J,Contribution #724435,NaN,161238.66,5735288468,NaN,NaN,Credit,2023-12-31 20:35:07-08:00
30383,12/31/2023,20:49:02,PST,Brian Osorio,Express Checkout Payment,Completed,USD,20.0,-1.19,18.81,spiked.arcadia@gmail.com,host+c6@opencollective.com,7SJ37588FN0150637,Non-Confirmed,0.0,4XB638642X6489318,Contribution #724441,NaN,161257.47,5735288468,NaN,NaN,Credit,2023-12-31 20:49:02-08:00


In [57]:
print(df_platform_paypal.value_counts('Balance Impact'))
print(df_platform_paypal.value_counts('Status'))
print(df_platform_paypal.value_counts('Type'))

Balance Impact
Credit    58265
Debit      1721
Name: count, dtype: int64
Status
Completed    59933
Pending         39
Unclaimed       14
Name: count, dtype: int64
Type
Subscription Payment        49762
Express Checkout Payment     8503
Mass Pay Payment             1607
Payment Refund                114
Name: count, dtype: int64


In [58]:
df_left_platform.sample(10)

,id,uuid,kind,payoutMethod,giftCardEmitterAccount,group,type,description,createdAt,merchantId,isRefunded,isRefund,refundTransaction,isOrderRejected,expense,__typename,amount.currency,amount.valueInCents,amount.__typename,amountInHostCurrency.currency,amountInHostCurrency.valueInCents,amountInHostCurrency.__typename,netAmount.currency,netAmount.valueInCents,netAmount.__typename,netAmountInHostCurrency.currency,netAmountInHostCurrency.valueInCents,netAmountInHostCurrency.__typename,paymentProcessorFee.currency,paymentProcessorFee.valueInCents,paymentProcessorFee.__typename,paymentMethod.name,paymentMethod.service,paymentMethod.sourcePaymentMethod,paymentMethod.type,account.id,account.legacyId,account.name,account.slug,account.isIncognito,account.type,account.__typename,oppositeAccount.id,oppositeAccount.legacyId,oppositeAccount.name,oppositeAccount.slug,oppositeAccount.isIncognito,oppositeAccount.type,oppositeAccount.__typename,permissions.id,permissions.canRefund,permissions.canDownloadInvoice,permissions.canReject,permissions.__typename,paymentMethod,expense.id,expense.type,expense.description,expense.invoiceInfo,expense.tags,expense.payee.id,expense.payee.name,expense.payee.slug,expense.payee.type,expense.payee.__typename,expense.__typename,refundTransaction.id,payoutMethod.type,payoutMethod.name,giftCardEmitterAccount.id,giftCardEmitterAccount.name,giftCardEmitterAccount.slug,giftCardEmitterAccount.__typename,amountInHostCurrency.value
835770,1c8abb73-d962-40e5-be15-2268c832b557,1c8abb73-d962-40e5-be15-2268c832b557,CONTRIBUTION,NaN,NaN,9fdff7a7-f352-47ac-b6a2-398b481500a5,CREDIT,Financial contribution to Colour Science,2023-01-01 06:58:49.365000+00:00,47M68880LM2286708,False,False,NaN,False,NaN,Credit,USD,6000,Amount,USD,6000,Amount,USD,5652,Amount,USD,5652,Amount,USD,-348,Amount,None,PAYPAL,NaN,PAYMENT,vjrkx5lm-nv904qjo-3az68bwa-7zdygoe3,67716.0,Colour Science,colour-science,False,COLLECTIVE,Collective,re0adkjr-v8xwm69m-8lkq7z5n-4l93bgoy,147222,Frederic Savoir,frederic-savoir,False,INDIVIDUAL,Individual,vedj9wro-z3a56ddv-lml67blg-8x4m0ykn,True,True,True,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.00
841889,77e10dbd-92ee-427c-9b8f-acefafd40a11,77e10dbd-92ee-427c-9b8f-acefafd40a11,CONTRIBUTION,NaN,NaN,9690b98c-cb34-470d-ba7f-5dd686c97d8c,CREDIT,Monthly financial contribution to Logseq (Back...,2023-01-01 01:51:42.053000+00:00,0EB13243A3523471Y,False,False,NaN,False,NaN,Credit,USD,500,Amount,USD,500,Amount,USD,426,Amount,USD,426,Amount,USD,-74,Amount,None,PAYPAL,NaN,SUBSCRIPTION,lk9mbw7y-48r3zq3m-gddp0ej5-lavnodgx,166914.0,Logseq,logseq,False,COLLECTIVE,Collective,gnxdwzj3-le5mpwjy-w90qy8rv-bko04a97,592061,Vladimito,vladimito,False,INDIVIDUAL,Individual,mvrwng4k-j03dpb8g-vjapz57o-yl9e8xba,True,True,True,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.00
843596,bca8c2fe-3835-451a-a497-e02b3f00e1eb,bca8c2fe-3835-451a-a497-e02b3f00e1eb,CONTRIBUTION,NaN,NaN,43910886-450f-49cd-9937-57e474498baa,CREDIT,Financial contribution to Sonarr,2023-01-01 01:01:40.995000+00:00,58846993058996421,False,False,NaN,False,NaN,Credit,USD,1000,Amount,USD,1000,Amount,USD,901,Amount,USD,901,Amount,USD,-99,Amount,None,PAYPAL,NaN,PAYMENT,5ax8emk7-lnw54q7g-vrmqgyvj-0o93zdrb,169319.0,Sonarr,sonarr,False,COLLECTIVE,Collective,jrkx5lmn-v904qjw0-vbkp8bwa-7zdygoe3,592040,skookum,guest-17603c89,False,INDIVIDUAL,Individual,vedj9wro-z3a56ddm-ok567blg-8x4m0ykn,True,True,True,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.00
799695,2bdeda8b-8b89-4891-bbcf-c7a42f1a3175,2bdeda8b-8b89-4891-bbcf-c7a42f1a3175,EXPENSE,NaN,NaN,8e047949-1b40-4566-a653-d2695d62c6fe,DEBIT,E-mail hosting,2023-01-11 00:18:37.184000+00:00,None,False,False,NaN,False,NaN,Debit,USD,-7860,Amount,USD,-7860,Amount,USD,-7860,Amount,USD,-7860,Amount,USD,0,Amount,NaN,NaN,NaN,NaN,rmvrwng4-kj03dpb9-v8x6z57o-yl9e8xba,36558.0,Piranha CMS,piranh

In [59]:
from utils import matchByDateAndValue

df_left_paypal = df_left_paypal[~df_left_paypal['Transaction ID'].isin(merged_df2['Transaction ID'])]
print(f"PayPal transactions with a corresponding Platform transaction: {len(merged_df2)}")
print(f"PayPal transactions without either a corresponding Platform transaction or a Bank transaction: {len(df_left_paypal)} out of {len(df_paypal_reconciliation)}")
print(f"Percentage of PayPal transactions that have a corresponding transaction either on the Platform or on the Bank: {round((len(merged_df2)+len(df_platform_paypal))/len(df_paypal_reconciliation)*100, 2)}%")
df_left_paypal.head(10)

PayPal transactions with a corresponding Platform transaction: 1
PayPal transactions without either a corresponding Platform transaction or a Bank transaction: 2264 out of 63431
Percentage of PayPal transactions that have a corresponding transaction either on the Platform or on the Bank: 94.57%


,Date,Time,TimeZone,Name,Type,Status,Currency,Gross,Fee,Net,From Email Address,To Email Address,Transaction ID,Address Status,Sales Tax,Reference Txn ID,Invoice Number,Receipt ID,Balance,Contact Phone Number,Subject,Note,Balance Impact,Datetime
157,08/28/2023,14:00:47,PDT,PayPal,Reversal of General Account Hold,Completed,USD,204.00,0.0,204.00,NaN,host+c6@opencollective.com,6A9882413N417000J,NaN,NaN,5L976115SX4006801,NaN,NaN,37139.79,NaN,Expense Payout for JHipster,NaN,Credit,2023-08-28 14:00:47-07:00
406,08/29/2023,09:01:13,PDT,PayPal,Reversal of General Account Hold,Completed,USD,918.00,0.0,918.00,NaN,host+c6@opencollective.com,3LL99812Y6482124M,NaN,NaN,5NV25925LD707793K,NaN,NaN,37549.83,NaN,Expense Payout for Material Theme,NaN,Credit,2023-08-29 09:01:13-07:00
410,08/29/2023,09:01:14,PDT,PayPal,Reversal of General Account Hold,Completed,USD,102.00,0.0,102.00,NaN,host+c6@opencollective.com,7WK88906SJ4003107,NaN,NaN,58X09818LP8492409,NaN,NaN,37142.58,NaN,Expense Payout for JHipster,NaN,Credit,2023-08-29 09:01:14-07:00
412,08/29/2023,09:01:14,PDT,PayPal,Reversal of General Account Hold,Completed,USD,346.80,0.0,346.80,NaN,host+c6@opencollective.com,81F28209HD241224P,NaN,NaN,98W55967ND3046931,NaN,NaN,37142.58,NaN,Expense Payout for Vuetify,NaN,Credit,2023-08-29 09:01:14-07:00
415,08/29/2023,09:01:14,PDT,PayPal,Reversal of General Account Hold,Completed,USD,357.00,0.0,357.00,NaN,host+c6@opencollective.com,3AA43889W2982094E,NaN,NaN,8TT121699A667845E,NaN,NaN,37077.33,NaN,Expense Payout for bootstrap-table,NaN,Credit,2023-08-29 09:01:14-07:00
421,08/29/2023,09:01:15,PDT,PayPal,Reversal of General Account Hold,Completed,USD,50.25,0.0,50.25,NaN,host+c6@opencollective.com,8HT60362KB078292W,NaN,NaN,90R99720U3262903N,NaN,NaN,36975.33,NaN,Expense Payout for Solus,NaN,Credit,2023-08-29 09:01:15-07:00
426,08/29/2023,09:01:16,PDT,PayPal,Reversal of General Account Hold,Completed,USD,65.25,0.0,65.25,NaN,host+c6@opencollective.com,9TY67436CY1024707,NaN,NaN,3GJ6209331237753T,NaN,NaN,36930.70,NaN,Expense Payout for The Organization for Ethica...,NaN,Credit,2023-08-29 09:01:16-07:00
428,08/29/2023,09:01:16,PDT,PayPal,Reversal of General Account Hold,Completed,USD,19.38,0.0,19.38,NaN,host+c6@opencollective.com,3BR42947BP1383405,NaN,NaN,4VV2657987607394U,NaN,NaN,36930.70,NaN,Expense Payout for REVDEV.world Mainnet Replac...,NaN,Credit,2023-08-29 09:01:16-07:00
430,08/29/2023,09:01:16,PDT,PayPal,Reversal of General Account Hold,Completed,USD,102.00,0.0,102.00,NaN,host+c6@opencollective.com,58M97813897128249,NaN,NaN,8UG76865VY846904B,NaN,NaN,36930.70,NaN,Expense Payout for Termux,NaN,Credit,2023-08-29 09:01:16-07:00
435,08/29/2023,09:01:17,PDT,PayPal,Reversal of General Account Hold,Completed,USD,25.25,0.0,25.25,NaN,host+c6@opencollective.com,608073666W773821V,NaN,NaN,7KK652884N3908406,NaN,NaN,36930.70,NaN,Expense Payout for Moto,NaN,Credit,2023-08-29 09:01:17-07:00


In [60]:
merged_df2.head()

,Date,Time,TimeZone,Name,Type,Status,Currency,Gross,Fee,Net,From Email Address,To Email Address,Transaction ID,Address Status,Sales Tax,Reference Txn ID,Invoice Number,Receipt ID,Balance,Contact Phone Number,Subject,Note,Balance Impact,Datetime,df2key,id,uuid,kind,payoutMethod,giftCardEmitterAccount,group,type,description,createdAt,merchantId,isRefunded,isRefund,refundTransaction,isOrderRejected,expense,__typename,amount.currency,amount.valueInCents,amount.__typename,amountInHostCurrency.currency,amountInHostCurrency.valueInCents,amountInHostCurrency.__typename,netAmount.currency,netAmount.valueInCents,netAmount.__typename,netAmountInHostCurrency.currency,netAmountInHostCurrency.valueInCents,netAmountInHostCurrency.__typename,paymentProcessorFee.currency,paymentProcessorFee.valueInCents,paymentProcessorFee.__typename,paymentMethod.name,paymentMethod.service,paymentMethod.sourcePaymentMethod,paymentMethod.type,account.id,account.legacyId,account.name,account.slug,account.isIncognito,account.type,account.__typename,oppositeAccount.id,oppositeAccount.legacyId,oppositeAccount.name,oppositeAccount.slug,oppositeAccount.isIncognito,oppositeAccount.type,oppositeAccount.__typename,permissions.id,permissions.canRefund,permissions.canDownloadInvoice,permissions.canReject,permissions.__typename,paymentMethod,expense.id,expense.type,expense.description,expense.invoiceInfo,expense.tags,expense.payee.id,expense.payee.name,expense.payee.slug,expense.payee.type,expense.payee.__typename,expense.__typename,refundTransaction.id,payoutMethod.type,payoutMethod.name,giftCardEmitterAccount.id,giftCardEmitterAccount.name,giftCardEmitterAccount.slug,giftCardEmitterAccount.__typename,amountInHostCurrency.value
29256,12/27/2023,17:05:31,PST,Hani Alhloul,Subscription Payment,Completed,USD,100.0,-5.48,94.52,support@likes.io,host+c6@opencollective.com,170798037F964922F,Non-Confirmed,0.0,I-G2GBC19JF9EW,NaN,NaN,149231.98,770800000,Monthly financial contribution to Dark Reader ...,NaN,Credit,2023-12-27 17:05:31-08:00,5015.0,f01df66f-a82c-48b3-808b-d3520b1f2611,f01df66f-a82c-48b3-808b-d3520b1f2611,CONTRIBUTION,NaN,NaN,3dc46c0a-1a4b-41b4-bb0f-8891556aa1da,CREDIT,Monthly financial contribution to Dark Reader ...,2023-12-28 01:05:58.641000+00:00,0XY0519281696092H,True,False,NaN,False,NaN,Credit,USD,10000.0,Amount,USD,10000.0,Amount,USD,9452.0,Amount,USD,9452.0,Amount,USD,-548.0,Amount,None,PAYPAL,NaN,SUBSCRIPTION,nmlo94zn-7x08dpox-7xpewga3-vjbrky5m,13775.0,Dark Reader,darkreader,False,COLLECTIVE,Collective,8k03reyd-5agmq50b-ydyplbwo-z7j4nxv9,743009.0,Buy Instagram Followers Stormlikes,stormlikes-net,False,INDIVIDUAL,Individual,3z8arxve-ymko60oa-oxa6gl5n-bj9w704d,True,True,True,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82cafecf-0a79-4c40-8c1f-19ee4f1106b6,NaN,NaN,NaN,NaN,NaN,NaN,100.0


In [61]:
df_paypal_bank = matchByDateAndValue(df_left_paypal, ['Datetime', 'Gross'], df_banks, ['Post Date', 'Debit'], '2 days', 0.05).dropna(subset=['df2key'])
df_left_paypal = df_left_paypal[~df_left_paypal['Transaction ID'].isin(df_paypal_bank['Transaction ID'])]
print(f"PayPal transactions with a corresponding Bank transaction: {len(df_paypal_bank)}")
print(f"PayPal transactions without either a corresponding Platform transaction or a Bank transaction: {len(df_left_paypal)} out of {len(df_paypal_reconciliation)}")
print(f"Percentage of PayPal transactions that have a corresponding transaction either on the Platform or on the Bank: {round((len(df_paypal_bank)+len(df_platform_paypal))/len(df_paypal_reconciliation)*100, 2)}%")
df_paypal_bank.head(10)

PayPal transactions with a corresponding Bank transaction: 43
PayPal transactions without either a corresponding Platform transaction or a Bank transaction: 2221 out of 63431
Percentage of PayPal transactions that have a corresponding transaction either on the Platform or on the Bank: 94.64%


,Date,Time,TimeZone,Name,Type,Status,Currency,Gross,Fee,Net,From Email Address,To Email Address,Transaction ID,Address Status,Sales Tax,Reference Txn ID,Invoice Number,Receipt ID,Balance,Contact Phone Number,Subject,Note,Balance Impact,Datetime,df2key,index,Post Date,Customer Ref,Debit,Credit,BAI Description,Detail,Account Number
410,08/29/2023,09:01:14,PDT,PayPal,Reversal of General Account Hold,Completed,USD,102.00,0.0,102.00,NaN,host+c6@opencollective.com,7WK88906SJ4003107,NaN,NaN,58X09818LP8492409,NaN,NaN,37142.58,NaN,Expense Payout for JHipster,NaN,Credit,2023-08-29 09:01:14-07:00,434.0,434.0,2023-08-28 00:00:00+00:00,C107375 JUSTWORK CO,99.00,NaN,PREAUTHORIZED ACH DEBIT,C107375 JUSTWORK COLLECTION 121B8D33AA...,4.869374e+09
435,08/29/2023,09:01:17,PDT,PayPal,Reversal of General Account Hold,Completed,USD,25.25,0.0,25.25,NaN,host+c6@opencollective.com,608073666W773821V,NaN,NaN,7KK652884N3908406,NaN,NaN,36930.70,NaN,Expense Payout for Moto,NaN,Credit,2023-08-29 09:01:17-07:00,437.0,437.0,2023-08-29 00:00:00+00:00,007000829105404,25.00,NaN,OUTGOING MONEY TRANSFER,BOB INTL WIRES ACCT#4869374308 ANDREW ASA...,4.869374e+09
2258,09/05/2023,10:00:54,PDT,PayPal,Reversal of General Account Hold,Completed,USD,2020.00,0.0,2020.00,NaN,host+c6@opencollective.com,9MC55528MX820571E,NaN,NaN,33Y862731P891041N,NaN,NaN,16113.97,NaN,Expense Payout for phalcon,NaN,Credit,2023-09-05 10:00:54-07:00,451.0,451.0,2023-09-06 00:00:00+00:00,007000906154008,2000.00,NaN,OUTGOING MONEY TRANSFER,BOB INTL WIRES ACCT#4869374308 DAMILOLA O...,4.869374e+09
5325,09/19/2023,11:01:16,PDT,PayPal,Reversal of General Account Hold,Completed,USD,10.02,0.0,10.02,NaN,host+c6@opencollective.com,7GH64455JH587703J,NaN,NaN,0LL60577KX224662V,NaN,NaN,83872.64,NaN,Expense Payout for DockSTARTer,NaN,Credit,2023-09-19 11:01:16-07:00,475.0,475.0,2023-09-21 00:00:00+00:00,007000921101947,10.00,NaN,OUTGOING MONEY TRANSFER,BOB INTL WIRES ACCT#4869374308 ANDREW ASA...,4.869374e+09
5394,09/19/2023,14:00:45,PDT,PayPal,Reversal of General Account Hold,Completed,USD,591.60,0.0,591.60,NaN,host+c6@opencollective.com,5BG14863S14543707,NaN,NaN,8SD06694NF0688306,NaN,NaN,79384.74,NaN,Expense Payout for AdNauseam,NaN,Credit,2023-09-19 14:00:45-07:00,473.0,473.0,2023-09-20 00:00:00+00:00,000000000000000,573.33,NaN,MISCELLANEOUS DEBIT,ANALYSIS ACTIVITY FOR 08/23,4.869374e+09
8934,10/03/2023,09:01:38,PDT,PayPal,Reversal of General Account Hold,Completed,USD,112.25,0.0,112.25,NaN,host+c6@opencollective.com,6ND095528W434522L,NaN,NaN,1GW51195T7676882G,NaN,NaN,71507.37,NaN,Expense Payout for Tarkov Development Community,NaN,Credit,2023-10-03 09:01:38-07:00,6.0,6.0,2023-10-05 00:00:00+00:00,007001005101425,111.00,NaN,OUTGOING MONEY TRANSFER,BOB INTL WIRES ACCT#4869374308 ANDREW ASA...,4.869374e+09
12137,10/17/2023,15:01:15,PDT,PayPal,Reversal of General Account Hold,Completed,USD,688.50,0.0,688.50,NaN,host+c6@opencollective.com,7W278445CS556063H,NaN,NaN,5GU11027T2633521H,NaN,NaN,72145.05,NaN,Expense Payout for AsyncAPI Mentorship 2023,NaN,Credit,2023-10-17 15:01:15-07:00,26.0,26.0,2023-10-19 00:00:00+00:00,007001019113134,675.00,NaN,OUTGOING MONEY TRANSFER,BOB INTL WIRES ACCT#4869374308 AISHAT MUI...,4.869374e+09
13545,10/24/2023,09:01:49,PDT,PayPal,Reversal of General Account Hold,Completed,USD,150.25,0.0,150.25,NaN,host+c6@opencollective.com,0PE4124733831444K,NaN,NaN,2H4570288E521293P,NaN,NaN,82080.48,NaN,Expense Payout for TrueCharts,NaN,Credit,2023-10-24 09:01:49-07:00,43.0,43.0,2023-10-26 00:00:00+00:00,C107375 JUSTWORK CO,147.89,NaN,PREAUTHORIZED ACH DEBIT,C107375 JUSTWORK COLLECTION 5A8DAF3122...,4.869374e+09
16855,11/07/2023,10:01:56,PST,PayPal,Reversal of General Account Hold,Completed,USD,4020.00,0.0,4020.00,NaN,host+c6@opencollective.com,8W336919US8027016,NaN,NaN,2KV2720042184833V,NaN,NaN,97563.83,NaN,Expense Payout for neovim,NaN,Credit,2023-11-07 10:01:56-08:00,59.0,59.0,2023-11-07 00:00:00+00:00,C107375 JUSTWORK CO,4178.46,NaN,PREAUTHORIZED ACH DEBIT,C107375 JUSTWORK COLLECTION A5213F3